In [10]:
#### importing dependencies

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Model, load_model, Sequential, model_from_yaml # to save our weights and load the pre-trained weights model
from keras.layers import Dense, Activation, Dropout, Input, Masking
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import random
import sys
import io

In [11]:
#### Importing the dataset

def read_data(path):
    print("Loading text data...")
    text = io.open(path, encoding='utf-8').read().lower()
    print("Done.")
    print("Corpus Length:", len(text))
    
    return text

In [12]:
text=read_data('threemenboat.txt')

Loading text data...
Done.
Corpus Length: 360663


In [13]:
#### Calculating the number of characters stored and building the c_to_i dictionary

chars=sorted(list(set(text)))

print('number of unique characters in the corpus:', len(chars))

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

Tx = 40 # sequence length

number of unique characters in the corpus: 60


In [14]:
#### Helper functions to build (train examples & labels) the dataset & vectorize it

def build_data(text, Tx = 40, steps = 3):
    """Create a training set by scanning a window of size Tx over the text corpus, with a stride of 3."""
    X = [] # training examples (list)
    Y = [] # training labels (list)
    for i in range(0, len(text) - Tx, steps):
        X.append(text[i: i + Tx])
        Y.append(text[i + Tx])
    print('number of training examples:', len(X))
    return X, Y


def vectorization(X, Y, n_x, char_indices, Tx = 40):
    """Convert X and Y (lists) into arrays to be given to a recurrent neural network."""
    
    m = len(X)
    x = np.zeros((m, Tx, n_x), dtype=np.bool) # array of shape (m, Tx, len(chars))
    y = np.zeros((m, n_x), dtype=np.bool) # array of shape (m, len(chars))
    for i, sentence in enumerate(X):
        for t, char in enumerate(sentence):
            x[i, t, char_indices[char]] = 1
        y[i, char_indices[Y[i]]] = 1
        
    return x, y 


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    out = np.random.choice(range(len(chars)), p = probas.ravel())
    return out
    

In [15]:
#### Building the data and Vectorizing it

X, Y = build_data(text, Tx=40, steps=3)
x, y = vectorization(X, Y, n_x = len(chars), char_indices = char_indices)


number of training examples: 120208


In [16]:
#### Defining the Model

print('Building model...')
model = Sequential()
model.add(LSTM(128, input_shape=(Tx, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Building model...


In [17]:
#### Function invoked at end of each epoch. Prints generated text.

def on_epoch_end(epoch, logs, maxlen=Tx):
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    diversity=0.4

    generated = ''
    sentence = text[start_index: start_index + maxlen]
    #usr_input = input("Write the beginning sentence and machine will complete it. Your input is: ")
    # zero pad the sentence to Tx characters.
    #sentence = ('{0:0>' + str(Tx) + '}').format(usr_input).lower()
    #generated += usr_input
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(1000):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
        
        if next_char == '\n':
            continue

In [18]:
#### Generating the output

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=30,
          callbacks=[print_callback])

Epoch 1/30
120208/120208 [==============================] - 52s 432us/step - loss: 2.0868

----- Generating text after Epoch: 0
----- Generating with seed: " supposed this was part of the method, a"
 supposed this was part of the method, and george the said we was not the could get in the ound you dice and we we light of dry with the ind out last from the with the want be the wast and the can a the thate the with a
that in the biges of at the grient the were that in the bant the bratter and the wast in a the want in the sind and the ind the erefine in the cally with the came fill or the wast in the can the badnt we say been the said in the batter that the and the baining a that a man a come and the could in the lite and the brean trout the the endered he said the wast in the was the sind a the boat the ead he had been in the was one now and had the gried and but the with the way out and he the been the lay the make that in see we see like firning the man the the pately intake the with 

dn’t i got housemaid’s knee?  why this in the late and some practisias and been beam for a tow-line, and stopped the barge of the case of the river and that the part of the sunging of the sail.

i have ease a mile and suir calling the little came of it.  the sight and tried to think the picnity as did not a few wet, and we precolled the side of the boat, and shade practice that i was the man who had gond the way of the boat and hang“ting the bag and river of the banks and the river.  i was something boat still before that it was a good plattion, and the churs shade a still, and the river.”

the call the boat, and asked himself, and the river, and the man was about his his much and streng the shallowed and stills and hearth, and the bank—and start of his time, and harris and we had it to get extraorned the sight of the river.  we went all that was hearth boats the boat and make his head, and had more and the boat, and it with a shallower the river.

                                     

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in log


party when i wanted a minute the fixe and bank and steady the point, and the carved of he had see a placest in the seemed more with the picture of the constity and beds you his head been strong, and he would be suirded the of anything to have been like the time part he would be to the faces, and the constite the seemed to the man who had to get to sleep to the man who had ever Epoch 18/30
120208/120208 [==============================] - 51s 420us/step - loss: 1.3080

----- Generating text after Epoch: 17
----- Generating with seed: " on the remainder.

six shillings a week"
 on the remainder.

six shillings a week, and then we were to still we were seemed to this exceptations, and then he would not nothing by the shout of the shout, and then it was a miluty and his land that the state of the small beer the boat and to start in the river thought and the still and rest that it was beer they said the two man was the stew and tried to the small be some ourse the other place and strange and

to be done would cone than the boat,” he said that we were the first lown the thing her part of the thing would say that we had been the boat,” he said that the pain aways the boat presentic the villing boat and get the middle of the sculls of the pickhep at the sune the boat and pleasant has the man which would con and so find harris, and the poor course, and see in a boat and see in up at the pilly the soat and half-past at the river the morning of some of the backwanted to be all relef”
sentury for the river of the river boat and so whish the boat, and then he was a trouther the shirts
the suspiction is the sudden that the chair, and spean and chook at the first beer the old way of it, and that i

In [19]:
# Saving weights and model

model_yaml = model.to_yaml()
with open("threemen_model.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)

model.save_weights("threemen_weights.h5")
print("Saved model to disk")


Saved model to disk
